In [1]:
import sys, os, pdb
import pandas as pd
import numpy as np
from datetime import date, timedelta
from dotenv import load_dotenv

load_dotenv()

from obility.agent import Agent
from obility.model import IndicatorList, KLine
from dichaos.agents.indexor.porfolio import Portfolio
from dichaos.kdutils import kd_logger

TA-Lib is not available: No module named 'talib'


In [2]:
symbol = 'RB'

In [3]:
### 构建模拟数据
n = 30
start_date = "2024-06-01"
trade_time = pd.date_range(start=start_date, periods=n, freq='B')
# 模拟价格数据
np.random.seed(42)
price = np.cumsum(np.random.randn(n)) + 100  # 模拟价格走势

open_price = price + np.random.randn(n) * 0.5
close_price = price + np.random.randn(n) * 0.5
high_price = np.maximum(open_price, close_price) + np.random.rand(n)
low_price = np.minimum(open_price, close_price) - np.random.rand(n)

# 模拟成交量和成交额
volume = np.random.randint(1000, 10000, size=n)
amount = volume * close_price

# 计算收益率（对数收益率）
returns = np.log(close_price / np.roll(close_price, 1))
returns[0] = 0  # 第一行无法计算对数收益率

# 构建 DataFrame
total_data = pd.DataFrame({
    'trade_time': trade_time,
    'open': open_price.round(2),
    'high': high_price.round(2),
    'low': low_price.round(2),
    'close': close_price.round(2),
    'volume': volume,
    'amount': amount.round(2),
    'return': returns.round(4)
})
total_data['code'] = symbol
total_data = total_data.set_index(['trade_time', 'code'])
total_data.tail()

,,open,high,low,close,volume,amount,return
trade_time,code,,,,,,,
2024-07-08,RB,96.49,96.79,95.55,95.77,5199,497920.41,0.0028
2024-07-09,RB,94.45,95.61,93.81,95.33,9445,900391.18,-0.0046
2024-07-10,RB,95.09,95.45,94.92,95.41,3557,339381.56,0.0009
2024-07-11,RB,94.81,95.42,93.69,94.38,6592,622168.83,-0.0109
2024-07-12,RB,94.84,95.35,94.23,94.61,1098,103884.23,0.0024


In [4]:
### 加载已经训练的agent
agent = Agent.load_checkpoint(path=os.path.join(os.environ['BASE_PATH'], 'memory', Agent.name, '2024-06-10'))

06-03 07:14 - Creating vector service: fassis
06-03 07:14 - Loading state_dict from records/memory/obility/2024-06-10/indexor/obility/brain/short_term_memory/state_dict.pkl
06-03 07:14 - Loading content_metadata from records/memory/obility/2024-06-10/indexor/obility/brain/short_term_memory/content_metadata.pkl
06-03 07:14 - Loading FAISS index for symbol 'RB' from records/memory/obility/2024-06-10/indexor/obility/brain/short_term_memory/RB.index
06-03 07:14 - Successfully reconstructed universe for 1 symbols.
06-03 07:14 - Creating embedding service: openai model: bge-m3
06-03 07:14 - Restored id_generator. Next ID will be: 2
06-03 07:14 - Creating vector service: fassis
06-03 07:14 - Loading state_dict from records/memory/obility/2024-06-10/indexor/obility/brain/mid_term_memory/state_dict.pkl
06-03 07:14 - Loading content_metadata from records/memory/obility/2024-06-10/indexor/obility/brain/mid_term_memory/content_metadata.pkl
06-03 07:14 - Successfully reconstructed universe for 0 sy

In [5]:
### 计算策略依赖的技术指标
rsi_df = agent.calculate_rsi(total_data)

macd_df = agent.calculate_macd(total_data)


macd_df.tail()

,,macd,signal,hist
trade_time,code,,,
2024-07-08,RB,-1.4107,-0.8750,-0.5357
2024-07-09,RB,-1.5090,-1.0018,-0.5072
2024-07-10,RB,-1.5624,-1.1139,-0.4485
2024-07-11,RB,-1.6686,-1.2249,-0.4438
2024-07-12,RB,-1.7145,-1.3228,-0.3917


In [6]:
## 用于回测
portfolio = Portfolio(symbol=symbol, lookback_window_size=0)

In [7]:

trade_time = '2024-06-14'

In [9]:
### K线数据
kline = KLine(date=trade_time,
                      symbol=symbol,
                      open=total_data.loc[(trade_time, symbol), 'open'],
                      close=total_data.loc[(trade_time, symbol), 'close'],
                      high=total_data.loc[(trade_time, symbol), 'high'],
                      low=total_data.loc[(trade_time, symbol), 'low'],
                      volume=total_data.loc[(trade_time, symbol), 'volume'])

print(kline.format())

2024-06-14 K线数据:
开盘价: 104.58
收盘价: 104.16
最高价: 105.44
最低价: 103.4
成交量: 7287.0



In [12]:
### 技术指标管理类
indicator_list = IndicatorList(date=trade_time)
indicator_list.set_indicator(rsi=rsi_df.loc[trade_time],
                             macd=macd_df.loc[trade_time])

print(indicator_list.format())

2024-06-14 技术指标:
RSI14: 85.5839 
MACD: 0.9982 



In [13]:
### 存入记忆池
agent.handing_data(trade_time, symbol, indicator_list, kline)

06-03 07:16 - {'text': '{"date":"2024-06-14","symbol":"RB","index":"-1","indicator":{"rsi":{"name":"RSI14","id":"rsi","values":85.5839},"macd":{"name":"MACD","id":"macd","values":0.9982},"date":"2024-06-14"},"kline":{"date":"2024-06-14","symbol":"RB","open":104.58,"close":104.16,"high":105.44,"low":103.4,"volume":7287.0}}', 'id': 2, 'important_score': 90.0, 'recency_score': 1.0, 'delta': 0, 'important_score_recency_compound_score': 1.9, 'access_counter': 0, 'date': '2024-06-14'}


In [14]:
### 获取记忆池中的数据
short_prompt, reflection_prompt = agent.query_records(trade_time, symbol)

In [16]:
print(reflection_prompt)

过去反思记忆索引ID:R1  内容:根据短期记忆中的技术指标分析，$RB出现下跌-0.0109的原因可能是由于RSI14值为33.7691，虽然未进入超卖区域，但接近中轴下方，显示出市场买方力量不足。同时，MACD值为-1.6686，为负值且未出现金叉，表明下跌动能仍然存在。此外，收盘价从94.81下跌至94.38，且最低价达到93.69，显示出卖方力量在增强。这些因素共同作用，可能导致了下个交易日的下跌。
过去反思记忆索引ID:R0  内容:根据短期记忆中的技术指标分析，$RB出现上涨0.0009的原因可能是由于RSI14值为28.0967，处于超卖区域，表明市场可能已经过度卖出，存在反弹的需求。同时，MACD值为-1.5624，虽然为负值，但如果出现金叉或柱状图能量减弱，可能预示着下跌动能减弱，上涨动能开始积累。此外，收盘价从95.09上涨至95.41，且最高价达到95.45，显示出买方力量在增强。这些因素共同作用，可能导致了下个交易日的上涨。



In [15]:
print(short_prompt)

短期记忆索引ID: S2
2024-06-14 技术指标:
RSI14: 85.5839 
MACD: 0.9982 
2024-06-14 K线数据:
开盘价: 104.58
收盘价: 104.16
最高价: 105.44
最低价: 103.4
成交量: 7287.0



In [17]:
response = agent.generate_prediction(
            date='trade_time',
            symbol=symbol,
            short_prompt=short_prompt,
            reflection_prompt=reflection_prompt)

06-03 07:19 - 

【核心决策要素】(请综合评估，寻找强共振信号)
    1. 评估MACD状态： 检查金叉/死叉情况、柱状图能量、与零轴的相对位置以及是否存在背离。
    2. 检查RSI水平： 判断是否处于超买/超卖区、是否接近50中轴、以及是否存在背离。
    3. 寻找信号共振点： 当MACD发出买入（卖出）信号，同时RSI也处于有利位置（如从超卖区向上，或突破50）时，买入（卖出）的确定性更高。
    4. 警惕背离信号： 出现顶背离时，即使价格仍在上涨，也应警惕回调风险，考虑减仓或设置止损；出现底背离时，即使价格仍在下跌，也可开始关注潜在的反弹机会。
    5. 结合周期分析： 短期信号需服从长期趋势。可参考不同时间周期的MACD和RSI表现，以提高决策准确性。


以下是短期记忆:
短期记忆索引ID: S2
2024-06-14 技术指标:
RSI14: 85.5839 
MACD: 0.9982 
2024-06-14 K线数据:
开盘价: 104.58
收盘价: 104.16
最高价: 105.44
最低价: 103.4
成交量: 7287.0


以下是过去反思记忆:
过去反思记忆索引ID:R1  内容:根据短期记忆中的技术指标分析，$RB出现下跌-0.0109的原因可能是由于RSI14值为33.7691，虽然未进入超卖区域，但接近中轴下方，显示出市场买方力量不足。同时，MACD值为-1.6686，为负值且未出现金叉，表明下跌动能仍然存在。此外，收盘价从94.81下跌至94.38，且最低价达到93.69，显示出卖方力量在增强。这些因素共同作用，可能导致了下个交易日的下跌。
过去反思记忆索引ID:R0  内容:根据短期记忆中的技术指标分析，$RB出现上涨0.0009的原因可能是由于RSI14值为28.0967，处于超卖区域，表明市场可能已经过度卖出，存在反弹的需求。同时，MACD值为-1.5624，虽然为负值，但如果出现金叉或柱状图能量减弱，可能预示着下跌动能减弱，上涨动能开始积累。此外，收盘价从95.09上涨至95.41，且最高价达到95.45，显示出买方力量在增强。这些因素共同作用，可能导致了下个交易日的上涨。


你不是要预测未来，而是要根据当前信息和既定规则，做出当下最优的概率决策。

必须严格按照以下JSON格式返回

In [18]:
response.__dict__

{'short_memory_index': 'S2',
 'reflection_memory_index': 'R1,R0',
 'reasoning': '当前RSI14值为85.5839，处于超买区域，表明市场可能已经过度买入，存在回调的需求。MACD值为0.9982，虽然为正值，但考虑到RSI的超买状态，上涨动能可能开始减弱。此外，收盘价从104.58下跌至104.16，且最低价达到103.4，显示出卖方力量在增强。这些因素共同作用，可能导致下个交易日的回调。',
 'confidence': 75.0,
 'signal': 'bearish',
 'analysis_details': 'RSI14: 85.5839 (超买区域), MACD: 0.9982 (正值但上涨动能可能减弱), 收盘价从104.58下跌至104.16, 最低价103.4。'}

In [21]:
future_data = total_data.loc[trade_time]
future_data = future_data.to_dict(orient='records')[0]

In [22]:
### 更新市场信息
portfolio.update_market_info(
            cur_date=trade_time,
            market_price=total_data.loc[trade_time, symbol]['close'],
            rets=total_data.loc[trade_time, symbol]['return'])

In [27]:
actions = agent.actions(response=response, threshold=80)
portfolio.record_action(action={'direction': actions})
actions

0

In [28]:
feedback = portfolio.feedback()
feedback

{'feedback': 0, 'date': '2024-06-14'}